# 目录

In [ ]:
1. requests
2. re
3. xpath
4. 模拟登录
5. 打码平台
6. ip代理

# 细节问题

In [2]:
"""
遇到的问题：
   UA
   Referer
   Cookie
   编码问题
   通过XHR获取数据
   json数据
   m3u8 + ts
   application/x-javascript转为json
   ts文件无法用windows自带的播放器播放
   ffmpeg
   各种音频格式相互换
   ts转MP4
   视频转音频
"""

'\n注意事项：\n   UA\n   Referer\n   Cookie\n   编码问题\n   通过XHR获取数据\n   json数据\n   m3u8 + ts\n   application/x-javascript转为json\n   ts文件无法用windows自带的播放器播放\n'

# Tips

In [10]:
# 对于数据量大的 如视频的爬取 可以先获取视频下载地址
# 记得保存

In [ ]:
# 多线程加速很好用

In [ ]:
# 先获取视频或者音频的下载地址后 多线程集中下载会比较好
# 也容易观测进度

# 爬虫耗时最大的是下载数据所需时间而不是分析页面

# UA

In [6]:
# 这个应该不用多说

# Referer

In [7]:
# 有时候除了UA之外还需要referer信息
# 如懒人听书
# 讽刺的是当你发现需要referer后 得到的音频下载地址任然只有部分有效

# Cookie

In [ ]:
# 对于需要登录的那些网站 不妨试试Cookie
# 不过需要注意cookie的时效性

# 所返回的json数据无法解析

In [1]:
# 获取到的json数据无法使用response.json()解析
# 错误原因 该网站返回的数据不符合json的格式
# 解决方案
page_text = getResponse(url).text
sonObj = demjson.decode(page_text[42:-2])
# 选取所返回的数据中数据json的部分 使用demjson解析

In [ ]:
# 返回的时json列表 
# 解析示例
def getMoreList(url):
    res = getResponse(url)
    temp = res.text
    jsonList = json.loads(temp)
    linkList = []
    for obj in jsonList:
        pageUrl = 'https://www.yicai.com' + obj['url']
        #print(pageUrl)
        linkList.append(pageUrl)
    return linkList

# requests获取的网页和浏览器中看到的不一样

In [ ]:
# 一般第一步就需要分析我们通过requests获取的网页HTML代码是否和浏览器中看到的网页一样
# 如果不一样 
# 最直接的办法就是使用selenium模拟

# 文本爬取

In [2]:
# 一般步骤
# 使用浏览器HTML分析工具选取所需文本对应区域即可
# 可用方式
# bs4、re、xpath

In [3]:
# 常见问题
# 编码问题
# 解决方案举例
"""
response = requests.get(...)
response.encoding = 'utf-8'
"""

"\nresponse = requests.get(...)\nresponse.encoding = 'utf-8'\n"

# 音频爬取

In [4]:
# 查找音频
# 浏览器分析工具选取
# 浏览器分析工具 ctrl+F 搜索 mp3 wav m4a

# 首先查看音频播放地址是否可以直接在HTML代码中找到
# 如果可以找到 提取 下载即可

# 注意：HTML中存在的播放地址可能不完整 需要自行根据各个网站添加一部分

# 找不到
# 分析网站是否是通过XHR请求获取的音频地址
# 打开浏览器中网页分析工具 NetWork部分 查看每一个请求以及收到的数据

# 最后 可能你找了半天得到的地址还是假的 
# 所以下载到的音频最好打开测试一下是否有效 
# 此条适用于视频

# 视频爬取

In [5]:
# 查找视频
# 寻找 mp4
# 同音频爬取



In [8]:
# 部分网站采用的时 ts 流媒体形式
# 需要通过m3u8文件下载每一个ts文件再进行文件合并
# 可以参考爬取cctv的例子 如cctv 经济半小时
#

# 各种不同格式的音频、视频的转换问题

In [9]:
# 可以在转换文件夹看到
# 可以通过多线程加速转换

# 隐藏selenium产生的浏览器窗口

In [ ]:
# 示例
def getM3U8(url):
    # 如下
    chrome_options =Options()
    chrome_options.add_argument('--headless')
    browser = webdriver.Chrome(options=chrome_options)
    #browser = webdriver.Chrome()
    browser.get(url)
    html = browser.page_source
    tree = etree.HTML(html)
    sourseUrl = tree.xpath('//*[@id="myFlash_player_html5_api"]/source/@src')
    if len(sourseUrl) == 0:
        return ""
    return sourseUrl[0]

# selenium获取的page_source与浏览器看到的还是不一样

In [ ]:
# 可能的解决方案是在获取page_source前sleep几秒
url = 'https://v.jin10.com/details.html?id=14225'
def getVideoSrc(url):
    chrome_options =Options()
    chrome_options.add_argument('--headless')
    browser = webdriver.Chrome(options=chrome_options)
    browser.get(url)
    time.sleep(3)
    page_text = browser.page_source
    tree = etree.HTML(page_text)
    src = tree.xpath('//*[@id="vjs_video_3_html5_api"]/@src')
    if len(src) == 0:
        return ""
    return src[0]

# selenium切换iframe以及属性定位

In [ ]:
def getMp4Url(url):
    # 需要使用selenium 并且切换iframe 加上 属性定位
    chrome_options =Options()
    chrome_options.add_argument('--headless')
    # 如何实现让selenium规避被检测到的风险
    option = ChromeOptions()
    option.add_experimental_option('excludeSwitches', ['enable-automation'])
    browser = webdriver.Chrome(executable_path='C:/Users/Administrator/Desktop/chromedriver_win32/chromedriver.exe', chrome_options=chrome_options, options=option)
    #browser = webdriver.Chrome(executable_path='C:/Users/Administrator/Desktop/chromedriver_win32/chromedriver.exe')
    browser.maximize_window()
    browser.get(url)
    #time.sleep(1)
    iframe = browser.find_element_by_tag_name("iframe")
    browser.switch_to.frame(iframe)
    time.sleep(1)
    video = browser.find_element_by_tag_name("video")
    src = video.get_attribute('origin_src')
    browser.quit()
    return src

print(getMp4Url('https://mp.weixin.qq.com/s/q0IFSxAc3hMm9lp4-YnEmA'))

# selenium 验证码

In [ ]:
# 有的网站看起来是需要验证码 实际上只需要Enter即可
# 针对验证码不需要对错 子需要Enter即可的情况
# https://www.findlaw.cn/video/25990.html
def retry(url):
    try:
        print('retry')
        browser = webdriver.Chrome(executable_path='C:/Users/Administrator/Desktop/chromedriver_win32/chromedriver.exe')
        browser.get(url)
        time.sleep(2)
        """
        inputLine = browser.find_element_by_id('vgcode')
        inputLine.send_keys('1234')
        btn = browser.find_element_by_id('resubimt')
        time.sleep(20)
        btn.click()
        time.sleep(60)
        """
        ActionChains(browser).send_keys(Keys.ENTER).perform()
        time.sleep(2)
        browser.quit()
    except Exception as e:
        return

# 获取某个标签下所有文本内容

In [ ]:
# 使用xpath定位到该标签后
# 注意 //text()
# /text() 获取的只是第一个文本内容
# tree.xpath('//label//text()')

In [1]:
l = [1, 2, 3]
for i in l:
    i = 1
print(l)

[1, 2, 3]
